In [1]:
import pandas as pd
df = pd.read_csv('train.csv')

In [ ]:
#5 binary,10 nominal,6 ordinal
#null values
col_names= list(df.columns)
col_names
em=[]
nd = {}
# df.iloc[:,1]
for i in col_names:
    if sum(df[i].isnull())>0:
        nd[i] = sum(df[i].isnull())

In [ ]:
#unique values
ne ={}
for i in col_names:
    if sum(df[i].isnull())>0:
        ne[i] = len(df[i].unique())

In [2]:
#redundant columns to remove 
columns_to_use= list(df.columns)
col2remove= ["nom_5","nom_6","nom_7","nom_8","nom_9","ord_5"]
cols =[]
for i in columns_to_use:
    if i not in col2remove:
        cols.append(i)
df2 = pd.DataFrame()
for i in cols:
    df2[i] = df[i]

In [17]:
#replacing NA with the most frequent value in that column
import statistics
df2.bin_0.fillna(statistics.mode(df2.bin_0),inplace=True)
for i in cols:
    if sum(df2[i].isnull())!=0:
        df2[i].fillna(statistics.mode(df2[i]),inplace=True)

In [ ]:
df_dummy = df2.copy(deep=True)
dummy = pd.get_dummies(df_dummy['nom_0'],prefix = "nom_0")
df_dummy = df_dummy.merge(dummy,left_index=True,right_index=True)
df_dummy = df_dummy.drop(['nom_0'],axis = 1)


In [ ]:
from sklearn.model_selection import train_test_split
df_dummy.columns

df_x_d = df_dummy.iloc[:,0:86]
df_y_d = df_dummy.iloc[:,86]

x_train_d, x_test_d, y_train_d, y_test_d = train_test_split(df_x_d,df_y_d,
                                                    train_size=0.8,test_size=0.2,
                                                    random_state = 7)

In [ ]:
#one hot encoding
cols2 = ['nom_0',
         'nom_1',
         'nom_2',
         'nom_3',
         'nom_4',
         'ord_1',
         'ord_2',
         'ord_3',
         'ord_4',]
for i in cols2:
    dummy = pd.get_dummies(df_dummy[i],prefix = f"{i}")
    df_dummy = df_dummy.merge(dummy,left_index=True,right_index=True)
    df_dummy = df_dummy.drop([i],axis = 1)


In [ ]:
#reindexing to make sure the target column remains in the last
df2.ord_2.unique()
columns_titles = ['id', 'bin_0', 'bin_1', 'bin_2', 'bin_3', 'bin_4', 'ord_0', 'day',
       'month', 'nom_0_Blue', 'nom_0_Green', 'nom_0_Red',
       'nom_1_Circle', 'nom_1_Polygon', 'nom_1_Square', 'nom_1_Star',
       'nom_1_Trapezoid', 'nom_1_Triangle', 'nom_2_Axolotl', 'nom_2_Cat',
       'nom_2_Dog', 'nom_2_Hamster', 'nom_2_Lion', 'nom_2_Snake',
       'nom_3_Canada', 'nom_3_China', 'nom_3_Costa Rica', 'nom_3_Finland',
       'nom_3_India', 'nom_3_Russia', 'nom_4_Bassoon', 'nom_4_Oboe',
       'nom_4_Piano', 'nom_4_Theremin', 'ord_1_Contributor', 'ord_1_Expert',
       'ord_1_Grandmaster', 'ord_1_Master', 'ord_1_Novice',
       'ord_2_Boiling Hot', 'ord_2_Cold', 'ord_2_Freezing', 'ord_2_Hot',
       'ord_2_Lava Hot', 'ord_2_Warm', 'ord_3_a', 'ord_3_b', 'ord_3_c',
       'ord_3_d', 'ord_3_e', 'ord_3_f', 'ord_3_g', 'ord_3_h', 'ord_3_i',
       'ord_3_j', 'ord_3_k', 'ord_3_l', 'ord_3_m', 'ord_3_n', 'ord_3_o',
       'ord_4_A', 'ord_4_B', 'ord_4_C', 'ord_4_D', 'ord_4_E', 'ord_4_F',
       'ord_4_G', 'ord_4_H', 'ord_4_I', 'ord_4_J', 'ord_4_K', 'ord_4_L',
       'ord_4_M', 'ord_4_N', 'ord_4_O', 'ord_4_P', 'ord_4_Q', 'ord_4_R',
       'ord_4_S', 'ord_4_T', 'ord_4_U', 'ord_4_V', 'ord_4_W', 'ord_4_X',
       'ord_4_Y', 'ord_4_Z','target']
df_dummy=df_dummy.reindex(columns=columns_titles)

In [ ]:
#we will perform oversampling using SMOTE
# import imblearn
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(df_x_d,df_dummy.iloc[:,86])

In [ ]:
#gradient boosting without oversampling-default parameters-81.53
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc_model = gbc.fit(x_train_d,y_train_d)
pred_gb = gbc_model.predict(x_test_d)
accuracy_score(y_test_d, pred_gb, normalize=True, sample_weight=None)

In [ ]:
#gradient boosting without oversampling-tuned parameters-81.71
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
gbc_hyper = GradientBoostingClassifier(min_samples_split=500,min_samples_leaf=250,
                                max_features=24,learning_rate=0.2,subsample=0.8,
                                n_estimators=500,verbose=1,presort=True)
gbc_model_hyper = gbc_hyper.fit(x_train_d,y_train_d)
pred_gb_hyper = gbc_model_hyper.predict(x_test_d)
accuracy_score(y_test_d, pred_gb_hyper, normalize=True, sample_weight=None)

In [ ]:
#gradient boosting with oversampling-default parameters-81.47
x_s,y_s = sm.fit_resample(x_train_d,y_train_d)
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
gbc_over = GradientBoostingClassifier()
gbc_model_over = gbc_over.fit(x_s,y_s)
pred_gb_over = gbc_model_over.predict(x_test_d)
accuracy_score(y_test_d, pred_gb_over, normalize=True, sample_weight=None)

In [ ]:
#gradient boosting with oversampling-tuning parameters-81.7
# x_s,y_s = sm.fit_resample(x_train_d,y_train_d)
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
gbc_over_hyper = GradientBoostingClassifier(min_samples_split=500,min_samples_leaf=250,
                                max_features=24,learning_rate=0.2,subsample=0.8,
                                n_estimators=500,verbose=1,presort=True)
gbc_model_over_hyper = gbc_over_hyper.fit(x_s,y_s)
pred_gb_over_hyper = gbc_model_over_hyper.predict(x_test_d)
accuracy_score(y_test_d, pred_gb_over_hyper, normalize=True, sample_weight=None)

In [ ]:
#linear model prediction accuracy using full feature set-81.47
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm_model = lm.fit(df_x_d,df_y_d)
# pred_lm = lm_model.predict(x_test_d)

In [ ]:
pred_lm[pred_lm>0.5]=1
pred_lm[pred_lm<0.5]=0
pred_lm

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test_d, pred_lm, normalize=True, sample_weight=None)

In [ ]:
#inference-feature selection using Recursive feature elimination reduces the accuracy by a fraction
from sklearn.feature_selection import RFE
rfe_ml = RFE(lm,20)
#transforming training data to RFE format
X_rfe = rfe_ml.fit_transform(x_train_d,y_train_d)  
#test data transformation
X_rfe_test = rfe_ml.fit_transform(x_test_d,y_test_d)  

In [ ]:
#prediction using reduced feature set
rfe_model = lm_model.fit(X_rfe,y_train_d)
pred_lm_rfe = rfe_model.predict(X_rfe_test)
# X_rfe.shape
pred_lm_rfe[pred_lm_rfe>0.5]=1
pred_lm_rfe[pred_lm_rfe<0.5]=0
accuracy_score(y_test_d, pred_lm_rfe, normalize=True, sample_weight=None)

In [ ]:
#logistic model accuracy with full feature set-81.294
from sklearn.linear_model import LogisticRegression
Log_reg = LogisticRegression(penalty='l2', dual=False, tol=0.0001)
log_model = Log_reg.fit(df_x_d,df_y_d)
# pred_log = log_model.predict(x_test_d)

In [ ]:
accuracy_score(y_test_d, pred_log, normalize=True, sample_weight=None)

In [ ]:
#transforming data into 20 best features for logistic regression
rfe_log = RFE(Log_reg,20)
X_rfe_train_log = rfe_log.fit_transform(x_train_d,y_train_d)
X_rfe_test_log = rfe_log.fit_transform(x_test_d,y_test_d)  

In [ ]:
#logistic reg prediction using reduced feature set-no change in accuracy
rfe_model_log = rfe_log.fit(X_rfe_train_log,y_train_d)
pred_log_rfe = rfe_model_log.predict(X_rfe_test_log)
accuracy_score(y_test_d, pred_log_rfe, normalize=True, sample_weight=None)

In [ ]:
#perceptron classification accuracy same as Logistic-81.294
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=10, activation='relu', solver='adam', alpha=0.0001)
mlp_model = mlp.fit(x_train_d,y_train_d)
pred_mlp = mlp_model.predict(x_test_d)

In [ ]:
accuracy_score(y_test_d, pred_mlp, normalize=True, sample_weight=None)

In [ ]:
#Random forest model prediction accuracy using full feature set-81.44
from sklearn.ensemble import RandomForestClassifier
rf_m = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto')
rf_model = rf_m.fit(df_x_d,df_y_d)
# pred_rf = rf_model.predict(x_test_d)